In [12]:
!apt-get install -y tesseract-ocr
!pip install pdf2image pytesseract pandas openpyxl
!apt-get install -y poppler-utils
!pip install pdfplumber pandas openpyxl
!pip install pymupdf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.4 MB/s eta 0:00:00


In [33]:
import pandas as pd
import re
from pdfminer.high_level import extract_text
from google.colab import files

# Step 1: Upload PDF File
print("📂 Please upload your bank statement PDF...")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]  # Get uploaded filename
print(f"✅ File uploaded successfully: {pdf_path}")

# Step 2: Extract Raw Text from PDF (Using pdfminer)
print("\n🔍 Extracting text from the PDF...")
extracted_text = extract_text(pdf_path)

# Step 3: Remove Illegal Characters (Non-printable & Excel-incompatible)
def clean_text(text):
    text = re.sub(r'[\x00-\x1F\x7F-\x9F\u200b\u202a\u202c]', '', text)  # Remove control & invisible chars
    text = text.replace('\f', '')  # Remove form feed character
    return text

cleaned_text = clean_text(extracted_text)

# Step 4: Convert Cleaned Text Into a DataFrame (Each Line in a New Row)
extracted_lines = cleaned_text.split("\n")  # Split text into separate lines
df = pd.DataFrame({"Extracted Text": extracted_lines})  # Store each line in a new row

# Step 5: Save as CSV (UTF-8) First
csv_output = "/content/full_extracted_text.csv"
df.to_csv(csv_output, index=False, encoding="utf-8")

# Step 6: Reload CSV and Save as Excel
output_xlsx = "/content/full_extracted_text.xlsx"
df = pd.read_csv(csv_output)  # Read CSV again

with pd.ExcelWriter(output_xlsx, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Full_Extracted_Text", index=False)

print(f"✅ Full extracted text saved to {output_xlsx}")
files.download(output_xlsx)


📂 Please upload your bank statement PDF...


Saving test3.pdf to test3 (13).pdf
✅ File uploaded successfully: test3 (13).pdf

🔍 Extracting text from the PDF...
✅ Full extracted text saved to /content/full_extracted_text.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:

with pdfplumber.open("test3.pdf") as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                print(row)
